In [42]:
import pandas as pd
import altair as alt
import numpy as np

In [45]:
source = pd.read_csv('/Users/tracy/sf_trees/data/processed/analysis.csv')

In [46]:
source.head()

,Unnamed: 0,district,GEOID,tree_counts,POPULATION,HOUSING_BURDEN,MINORITY,SVI,SVI_rank,HOUSING_BURDEN_rank,...,area,trees_per_acre,HISTO_1,HISTO_NODA,median_income_x,canopy,neighborhood,SF Census Tract,Tree coverage,median_income_y
0,0,1.0,6075013300,36,4487.0,18.7,26.1,7.0071,0.5230,0.4318,...,143.643535,0.25062,74953.0,506358.0,$169.4K,12.9,Presidio Heights,133.00,12.9%,169392.0
1,1,1.0,6075013300,36,4487.0,18.7,26.1,7.0071,0.5230,0.4318,...,143.643535,0.25062,74953.0,506358.0,$169.4K,12.9,Presidio Heights,133.00,12.9%,169392.0
2,2,1.0,6075015401,17,3252.0,22.3,44.6,5.8468,0.3808,0.5620,...,96.949039,0.17535,38386.0,353959.0,$121.2K,9.8,Presidio Heights,154.01,9.8%,121250.0
3,3,1.0,6075015401,17,3252.0,22.3,44.6,5.8468,0.3808,0.5620,...,96.949039,0.17535,38386.0,353959.0,$121.2K,9.8,Presidio Heights,154.01,9.8%,121250.0
4,4,1.0,6075015401,17,3252.0,22.3,44.6,5.8468,0.3808,0.5620,...,96.949039,0.17535,38386.0,353959.0,$121.2K,9.8,Presidio Heights,154.01,9.8%,121250.0


In [7]:
# street trees per acre and median income

In [76]:
trees_nbh = source.groupby('neighborhood').agg(tree_counts=('tree_counts', 'sum'), 
                                              population=('POPULATION', 'sum'),
                                              area = ('area','sum'),
                                              median_income = ('median_income_y','median'))
trees_nbh['trees_per_acre'] = trees_nbh['tree_counts']/trees_nbh['area']


In [71]:
income_nbh = source.groupby('neighborhood').apply(
    lambda x: pd.Series({
    'weighted_median_income': (x['median_income_y'] * x['POPULATION']).sum() / x['POPULATION'].sum()
})
).reset_index()
income_nbh['weighted_median_income'] = income_nbh['weighted_median_income'].astype(int).apply(lambda x: round(x, 0))

In [77]:
trees_income = pd.merge(trees_nbh,income_nbh,on='neighborhood') 

In [96]:
trees_income['income_tag'] = (trees_income['weighted_median_income'].astype(float)/1000).apply(lambda x: round(x, 1)) 
trees_income['income_tag']  = '$' + trees_income['income_tag'] .astype(str) + 'K'

In [97]:
trees_income.head()

,neighborhood,tree_counts,population,area,median_income,trees_per_acre,weighted_median_income,income_tag
0,Bayview Hunters Point,11542,65972.0,7024.128203,79091.0,1.643193,78776,$78.8K
1,Bernal Heights,8301,90759.0,2278.918564,148891.5,3.642517,150889,$150.9K
2,Castro/Upper Market,5619,36107.0,897.237788,205420.0,6.262554,198738,$198.7K
3,Chinatown,624,14310.0,143.788092,30519.0,4.339720,32917,$32.9K
4,Excelsior,4407,56322.0,1325.442821,104406.0,3.324927,104109,$104.1K


In [121]:
base = alt.Chart(trees_income).mark_square(size=60).encode(
    alt.X('weighted_median_income',title="Median Household Income"),
    alt.Y('trees_per_acre',title='Number of Street Trees Per Acre'))
points= base.encode(    
    size=alt.Size('population',title='Population'),
    color = alt.Color('neighborhood',legend=None),
    
    tooltip=[alt.Tooltip('neighborhood',title='Neighborhood'),
             alt.Tooltip('population',title='Population'),
             alt.Tooltip('income_tag',title='Median Income')]
).interactive()
    
line = base.transform_regression('weighted_median_income','trees_per_acre').mark_line()

viz_2 = (points + line).properties(    
    width=500,
    height=350,
    title='Median Household Income vs. Street Tree Density, by Neighborhood')
viz_2

alt.LayerChart(...)

In [120]:
viz_2.save('viz_2.html')